In [1]:
import yfinance as yf
import pandas as pd
import requests
from urllib.parse import urlparse
from IPython.display import Image, display

# Replace with your actual Logo.dev API key
API_KEY = 'pk_Gq2oOGVWQpiDsXzAK1BOGA'


# FUNCTION 1) Make initial dataframe function:
def get_basicInfo(portfolio: dict) -> pd.DataFrame:
    ticker_symbols = []  # Get ticker symbols (e.g., 'AAPL')
    ticker_names = []  # Get company names (e.g., 'Apple Inc.')
    ticker_currencies = []  # Get instrument currency (USD, CAD)
    ticker_currentPrices = []  # Get current share price
    ticker_total_values = []  # List to store total value of each holding in base currency
    ticker_total_values_cad = []  # List to store total value in CAD
    ticker_logos = []  # List to store logo URLs for each ticker

    # Get USD to CAD exchange rate for conversion (from USD to CAD)
    usd_cad_rate = yf.Ticker('USDCAD=X').history(period="1d")['Close'].iloc[0]

    for ticker, share_count in portfolio.items():
        # GET COMPANY NAME & CURRENCY
        ticker_info = yf.Ticker(ticker).info  # access yfinance info
        ticker_symbols.append(ticker)  # Ticker symbol itself (e.g., 'AAPL')
        ticker_names.append(ticker_info['longName'])  # Full company name (e.g., 'Apple Inc.')
        ticker_currencies.append(ticker_info['currency'])

        # GET CURRENT SHARE PRICE
        tickerPrice = yf.Ticker(ticker).history(period="1d")['Close'].iloc[0]  # Latest closing price
        ticker_currentPrices.append(tickerPrice)

        # Compute the total value for the ticker (share count * price)
        total_value = share_count * tickerPrice
        ticker_total_values.append(total_value)

        # Convert total value to CAD if the currency is USD
        if ticker_info['currency'] == 'USD':
            total_value_cad = total_value * usd_cad_rate  # Convert to CAD
        else:
            total_value_cad = total_value  # If already in CAD, no conversion needed
        ticker_total_values_cad.append(total_value_cad)

        # Get the logo URL for the ticker (if available)
        domain_name = get_clean_domain(ticker)
        if domain_name:
            logo_url = get_logo_url(domain_name)
            ticker_logos.append(logo_url if logo_url else None)
        else:
            ticker_logos.append(None)

    # Create DataFrame with all information
    dataDict = {
        'Ticker': ticker_symbols,  # Add ticker symbols to the DataFrame
        'Ticker Name': ticker_names,
        'Currency': ticker_currencies,
        'Current Price': ticker_currentPrices,
        'Total Value (Base)': ticker_total_values,
        'Total Value (CAD)': ticker_total_values_cad,
        'Logo URL': ticker_logos  # Add logo URLs
    }

    df_dataDict = pd.DataFrame(dataDict)

    # Sort by 'Total Value (CAD)' for better readability
    df_dataDict = df_dataDict.sort_values(by='Total Value (CAD)', ascending=False)

    # Return DataFrame without formatting for now
    return df_dataDict


# FUNCTION 2) Update ticker prices
def update_ticker_prices(df_portfolio: pd.DataFrame, 
                        price_updates: dict, portfolio: dict, 
                        usd_cad_rate: float, currency_updates: dict) -> pd.DataFrame:
    """
    Update the current price of multiple tickers in the dataframe.
    
    Parameters:
    - df_portfolio: DataFrame containing the portfolio data
    - price_updates: A dictionary where keys are ticker symbols and values are the new prices
    - portfolio: The portfolio dictionary with share counts
    - usd_cad_rate: The current USD to CAD exchange rate
    - currency_updates: A dictionary where keys are ticker symbols and values are the currency of the updated price ('USD' or 'CAD')
    
    Returns:
    - Updated DataFrame with the new prices
    """
    # Pre-fetch ticker information for all tickers (once per update)
    tickers_info = {ticker: yf.Ticker(ticker).info for ticker in price_updates.keys()}

    for ticker, new_price in price_updates.items():
        # Locate the ticker by 'Ticker' column and update the 'Current Price'
        if ticker in df_portfolio['Ticker'].values:
            df_portfolio.loc[df_portfolio['Ticker'] == ticker, 'Current Price'] = new_price

            # Get the share count from the original portfolio
            share_count = portfolio[ticker]  

            # Determine the currency of the updated price (USD or CAD)
            price_currency = currency_updates.get(ticker, 'USD')  # Default to USD if not provided
            
            # Compute the total value in base currency (price * shares)
            total_value = share_count * new_price
            
            # Convert the total value to CAD if necessary
            if price_currency == 'USD':
                total_value_cad = total_value * usd_cad_rate  # Convert to CAD
            else:
                total_value_cad = total_value  # If already in CAD, no conversion needed

            # Update the respective columns
            df_portfolio.loc[df_portfolio['Ticker'] == ticker, 'Total Value (Base)'] = total_value
            df_portfolio.loc[df_portfolio['Ticker'] == ticker, 'Total Value (CAD)'] = total_value_cad

    return df_portfolio


# Helper function to extract domain name (without 'https://', 'http://', or 'www.')
def get_clean_domain(ticker_symbol):
    """Extract and clean the domain from a company's ticker symbol."""
    ticker = yf.Ticker(ticker_symbol)
    company_info = ticker.info
    website_url = company_info.get('website', 'Website not available')
    
    if website_url != 'Website not available':
        # Parse the URL to get the domain without 'https://', 'http://', or 'www.'
        parsed_url = urlparse(website_url)
        return parsed_url.netloc.replace('www.', '')
    return None


# Helper function to fetch the logo from Logo.dev API
def get_logo_url(domain_name):
    """Fetch the logo for a given domain using the Logo.dev API."""
    url = f"https://img.logo.dev/{domain_name}?token={API_KEY}&size=200&format=png"
    response = requests.get(url)
    
    if response.status_code == 200:
        return url
    return None


# Example Portfolio Data
portfolio = {
    'AAPL': 17.8658, 'ADBE': 10,
    'ATD.TO': 9, 'CCO.TO': 9.3874,
    'CNQ.TO': 10, 'COST': 44.7886,
    'CVE': 23, 'ERO.TO': 32.036,
    'FAST': 5.7558, 'FCX': 16, 'K.TO': 8.5485,
    'MATR.TO': 46, 'MCO': 0.5636, 'META': 13.5995,
    'MSFT': 52, 'NFLX': 1.6523, 'NTR': 8.2221,
    'NVDA': 60, 'PG': 7, 'PHYS': 1.4343,
    'QBR-B.TO': 12, 'RKLB': 88,
    'RMD': 1.3009, 'TECK-A.TO': 4.2728,
    'TSM': 7.874, 'U': 22.9426,
    'UEC': 32, 'URNM': 12,
    'VFV.TO': 27.4385, 'V': 13,
    'XID.TO': 5, 'BTC-USD': 0.03070794,
    'RYCEY': 12, 'HTHIY': 4
}

# Fetch the USD to CAD exchange rate
usd_cad_rate = yf.Ticker('USDCAD=X').history(period="1d")['Close'].iloc[0]

# Call the initial function to get the portfolio information
df_data = get_basicInfo(portfolio)

# Example: New dict of tickers and their updated prices
price_updates = {
    'AAPL': 34.43,
    'ADBE': 20.98,
    'COST': 45.05,
    'META':31.86,
    'MSFT':31.26,
    'NVDA':32.02,
    'PG':30.74,
    'V':29.59,
}

# Currency of the updated price (either 'USD' or 'CAD')
currency_updates = {
    'AAPL': 'CAD',
    'ADBE': 'CAD',
    'COST': 'CAD',
    'META':'CAD',
    'MSFT':'CAD',
    'NVDA':'CAD',
    'PG':'CAD',
    'V':'CAD',
}

# Update the DataFrame with the new prices
df_data_updated = update_ticker_prices(df_data, price_updates, portfolio, usd_cad_rate, currency_updates)

# Sort the DataFrame by 'Total Value (CAD)' numerically (before formatting)
df_data_updated = df_data_updated.sort_values(by='Total Value (CAD)', ascending=False)

# Format the numeric columns as currency (USD and CAD) after sorting
df_data_updated['Current Price'] = df_data_updated['Current Price'].apply(lambda x: f"${x:,.2f}")
df_data_updated['Total Value (Base)'] = df_data_updated['Total Value (Base)'].apply(lambda x: f"${x:,.2f}")
df_data_updated['Total Value (CAD)'] = df_data_updated['Total Value (CAD)'].apply(lambda x: f"${x:,.2f}")

# Display the updated DataFrame with the logos (or URLs if using in a Jupyter environment)
df_data_updated[['Ticker Name', 'Total Value (CAD)', 'Logo URL']]


ModuleNotFoundError: No module named 'yfinance'

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from urllib.parse import urlparse
from IPython.display import Image, display

In [ ]:
API_KEY = 'pk_Gq2oOGVWQpiDsXzAK1BOGA' # for Logo.dev

In [ ]:
###################################################################################################
# FUNCTION 1) Make initial dataframe function:
###################################################################################################

def get_basicInfo(portfolio: dict) -> pd.DataFrame:

    ticker_symbols = []  # Get ticker symbols (e.g., 'AAPL')
    ticker_names = []  # Get company names (e.g., 'Apple Inc.')
    ticker_currencies = []  # Get instrument currency (USD, CAD)
    ticker_currentPrices = []  # Get current share price
    ticker_total_values = []  # List to store total value of each holding in base currency
    ticker_total_values_cad = []  # List to store total value in CAD
    ticker_logos = []  # List to store logo URLs for each ticker

    # Get USD to CAD exchange rate for conversion (from USD to CAD)
    usd_cad_rate = yf.Ticker('USDCAD=X').history(period="1d")['Close'].iloc[0]

    for ticker, share_count in portfolio.items():
        # GET COMPANY NAME & CURRENCY
        ticker_info = yf.Ticker(ticker).info  # access yfinance info
        ticker_symbols.append(ticker)  # Ticker symbol itself (e.g., 'AAPL')
        ticker_names.append(ticker_info['longName'])  # Full company name (e.g., 'Apple Inc.')
        ticker_currencies.append(ticker_info['currency'])

        # GET CURRENT SHARE PRICE
        tickerPrice = yf.Ticker(ticker).history(period="1d")['Close'].iloc[0]  # Latest closing price
        ticker_currentPrices.append(tickerPrice)

        # Compute the total value for the ticker (share count * price)
        total_value = share_count * tickerPrice
        ticker_total_values.append(total_value)

        # Convert total value to CAD if the currency is USD
        if ticker_info['currency'] == 'USD':
            total_value_cad = total_value * usd_cad_rate  # Convert to CAD
        else:
            total_value_cad = total_value  # If already in CAD, no conversion needed
        ticker_total_values_cad.append(total_value_cad)

        # Get the logo URL for the ticker (if available)
        domain_name = get_clean_domain(ticker)
        if domain_name:
            logo_url = get_logo_url(domain_name)
            ticker_logos.append(logo_url if logo_url else None)
        else:
            ticker_logos.append(None)

    # Create DataFrame with all information
    dataDict = {
        'Ticker': ticker_symbols,  # Add ticker symbols to the DataFrame
        'Ticker Name': ticker_names,
        'Currency': ticker_currencies,
        'Current Price': ticker_currentPrices,
        'Total Value (Base)': ticker_total_values,
        'Total Value (CAD)': ticker_total_values_cad,
        'Logo URL': ticker_logos  # Add logo URLs
    }

    df_dataDict = pd.DataFrame(dataDict)

    # Sort by 'Total Value (CAD)' for better readability
    df_dataDict = df_dataDict.sort_values(by='Total Value (CAD)', ascending=False)

    # Return DataFrame without formatting for now
    return df_dataDict




###################################################################################################
# FUNCTION 2) Update ticker prices
###################################################################################################

def update_ticker_prices(df_portfolio: pd.DataFrame, 
                        price_updates: dict, portfolio: dict, 
                        usd_cad_rate: float, currency_updates: dict) -> pd.DataFrame:
    """
    Update the current price of multiple tickers in the dataframe.
    
    Parameters:
    - df_portfolio: DataFrame containing the portfolio data
    - price_updates: A dictionary where keys are ticker symbols and values are the new prices
    - portfolio: The portfolio dictionary with share counts
    - usd_cad_rate: The current USD to CAD exchange rate
    - currency_updates: A dictionary where keys are ticker symbols and values are the currency of the updated price ('USD' or 'CAD')
    
    Returns:
    - Updated DataFrame with the new prices
    """
    # Pre-fetch ticker information for all tickers (once per update)
    tickers_info = {ticker: yf.Ticker(ticker).info for ticker in price_updates.keys()}

    for ticker, new_price in price_updates.items():
        # Locate the ticker by 'Ticker' column and update the 'Current Price'
        if ticker in df_portfolio['Ticker'].values:
            df_portfolio.loc[df_portfolio['Ticker'] == ticker, 'Current Price'] = new_price

            # Get the share count from the original portfolio
            share_count = portfolio[ticker]  

            # Determine the currency of the updated price (USD or CAD)
            price_currency = currency_updates.get(ticker, 'USD')  # Default to USD if not provided
            
            # Compute the total value in base currency (price * shares)
            total_value = share_count * new_price
            
            # Convert the total value to CAD if necessary
            if price_currency == 'USD':
                total_value_cad = total_value * usd_cad_rate  # Convert to CAD
            else:
                total_value_cad = total_value  # If already in CAD, no conversion needed

            # Update the respective columns
            df_portfolio.loc[df_portfolio['Ticker'] == ticker, 'Total Value (Base)'] = total_value
            df_portfolio.loc[df_portfolio['Ticker'] == ticker, 'Total Value (CAD)'] = total_value_cad

    return df_portfolio


# Helper function to extract domain name (without 'https://', 'http://', or 'www.')
def get_clean_domain(ticker_symbol):
    """Extract and clean the domain from a company's ticker symbol."""
    ticker = yf.Ticker(ticker_symbol)
    company_info = ticker.info
    website_url = company_info.get('website', 'Website not available')
    
    if website_url != 'Website not available':
        # Parse the URL to get the domain without 'https://', 'http://', or 'www.'
        parsed_url = urlparse(website_url)
        return parsed_url.netloc.replace('www.', '')
    return None


# Helper function to fetch the logo from Logo.dev API
def get_logo_url(domain_name):
    """Fetch the logo for a given domain using the Logo.dev API."""
    url = f"https://img.logo.dev/{domain_name}?token={API_KEY}&size=200&format=png"
    response = requests.get(url)
    
    if response.status_code == 200:
        return url
    return None

# Function to calculate the 1-year return for a portfolio
def get_1y_return(portfolio: dict) -> dict:
    """
    Calculate the 1-year return for each ticker in the portfolio.

    Parameters:
    - portfolio: A dictionary where keys are ticker symbols and values are the number of shares.

    Returns:
    - A dictionary where keys are ticker symbols and values are the 1-year return percentages.
    """
    return_dict = {}
    
    for ticker in portfolio:
        ticker_data = yf.Ticker(ticker)
        hist = ticker_data.history(period="1y")  # Get 1-year historical data
        
        # Check if there is enough data
        if len(hist) > 0:
            start_price = hist['Close'].iloc[0]  # Price 1 year ago
            end_price = hist['Close'].iloc[-1]   # Most recent price
            
            # Calculate the percentage change
            return_percentage = ((end_price - start_price) / start_price) * 100
            return_dict[ticker] = return_percentage
        else:
            return_dict[ticker] = None  # Handle case where data is not available

    return return_dict

In [ ]:
# Example Portfolio Data
portfolio = {

    ### Wealthsimple TFSA & (FHSA) ###
    'AAPL': 17.8658, 'ADBE': 10,
    'ATD.TO': 9, 'CCO.TO': 9.3874,
    'CNQ.TO': 9+(0.2347), 'COST': 26.7886,
    # 'CVE': 0 (SOLD),
    'EBIT.TO': 7+(4), 'ERO.TO': 32.036+(1),
    'FAST': 7.7728, 'FCX': 16, 
    'K.TO': 11.5485+(2), 'LEU': 6,
    'MATR.TO': 39, 
    'MCO': 0.5636, 'META': 13.5995,
    'MOD': 8, 'MSFT': 33, 
    'NFLX': 1.6782, 'NTR': 3.2221,
    'NVDA': 35, 'PG': 7, 
    'PHYS': 1.4343+(1), # 'QBR-B.TO': 0 (SOLD), 
    'RKLB': 207+(25), 'RMD': 1.3009, 
    'TECK-A.TO': 4.2728, 'TSM': 5.874,
    'U': 25.9426, 'UEC': 20, 
    'URNM': 3, 'VFV.TO': 21.4385, 
    'V': 13, # 'XID.TO': 0 (SOLD),

    # ### Wealthsimple TFSA ###
    # 'AAPL': 17.8658, 'ADBE': 10,
    # 'ATD.TO': 9, 'CCO.TO': 9.3874,
    # 'CNQ.TO': 9, 'COST': 26.7886,
    # # 'CVE': 0 (SOLD),
    # 'EBIT.TO': 7, 'ERO.TO': 32.036,
    # 'FAST': 7.7728, 'FCX': 16, 
    # 'K.TO': 11.5485, 'LEU': 6,
    # 'MATR.TO': 39, 
    # 'MCO': 0.5636, 'META': 13.5995,
    # 'MOD': 8, 'MSFT': 33, 
    # 'NFLX': 1.6782, 'NTR': 3.2221,
    # 'NVDA': 35, 'PG': 7, 
    # 'PHYS': 1.4343, # 'QBR-B.TO': 0 (SOLD), 
    # 'RKLB': 207, 'RMD': 1.3009, 
    # 'TECK-A.TO': 4.2728, 'TSM': 5.874,
    # 'U': 25.9426, 'UEC': 20, 
    # 'URNM': 3, 'VFV.TO': 21.4385, 
    # 'V': 13, # 'XID.TO': 0 (SOLD),

    ### Crypto.com ###
    'BTC-USD': 0.03070794,

    ### Interactive Brokers ###
    'RYCEY': 12, 'HTHIY': 4,

    ### Wealthsimple FHSA ###
    # 'CNQ.TO': 0.2347, 'EBIT.TO': 4,
    # 'ERO.TO': 1, 'K.TO': 2,
    # 'PHYS': 1, 'RKLB': 25,
}

In [ ]:
# Example: New dict of tickers and their updated prices
price_updates = {
    'AAPL': 34.43,
    'ADBE': 20.98,
    'COST': 45.05,
    'META':31.86,
    'MSFT':31.26,
    'NVDA':32.02,
    'PG':30.74,
    'V':29.59,
}

# Currency of the updated price (either 'USD' or 'CAD')
currency_updates = {
    'AAPL': 'CAD',
    'ADBE': 'CAD',
    'COST': 'CAD',
    'META':'CAD',
    'MSFT':'CAD',
    'NVDA':'CAD',
    'PG':'CAD',
    'V':'CAD',
}

In [ ]:

# Fetch the USD to CAD exchange rate
usd_cad_rate = yf.Ticker('USDCAD=X').history(period="1d")['Close'].iloc[0]

# Call the initial function to get the portfolio information
df_data = get_basicInfo(portfolio)

# Update the DataFrame with the new prices
df_data_updated = update_ticker_prices(df_data, price_updates, portfolio, usd_cad_rate, currency_updates)

# Sort the DataFrame by 'Total Value (CAD)' numerically (before formatting)
df_data_updated = df_data_updated.sort_values(by='Total Value (CAD)', ascending=False)

# Format the numeric columns as currency (USD and CAD) after sorting
df_data_updated['Current Price'] = df_data_updated['Current Price'].apply(lambda x: f"${x:,.2f}")
df_data_updated['Total Value (Base)'] = df_data_updated['Total Value (Base)'].apply(lambda x: f"${x:,.2f}")
df_data_updated['Total Value (CAD)'] = df_data_updated['Total Value (CAD)'].apply(lambda x: f"${x:,.2f}")

# Get 1y returns
return_dict = get_1y_return(portfolio)

print('----------------------------------------------------------------------------------')

# Display the logos in the notebook (for each ticker in the portfolio)
# Display the logos and additional information in the notebook (for each ticker in the portfolio)
for index, row in df_data_updated.iterrows():
    logo_url = row['Logo URL']
    if logo_url:
        display(Image(url=logo_url, width=100, height=100))  # This will display the image in the notebook
    
    # Get the 1-year return for the ticker
    ticker_name = row['Ticker Name']
    ticker_symbol = row['Ticker']
    total_value_cad = row['Total Value (CAD)']
    
    # Get 1-year return from return_dict, using ticker_symbol to match
    one_year_return = return_dict.get(ticker_symbol, None)
    
    # If return is None, display 'No data' for that ticker's 1-year return
    if one_year_return is not None:
        one_year_return_str = f"{one_year_return:.2f}%"
    else:
        one_year_return_str = 'No data available'

    # Print the information for each ticker
    print(f"Ticker Name: {ticker_name}")
    print(f"Total Value (CAD): {total_value_cad}")
    print(f"1-Year Return: {one_year_return_str}")
    print('----------------------------------------------------------------------------------')
# The updated DataFrame with logos is ready!
    
    df_data_updated

----------------------------------------------------------------------------------


Ticker Name: Rocket Lab USA, Inc.
Total Value (CAD): $7,866.06
1-Year Return: 443.15%
----------------------------------------------------------------------------------
Ticker Name: Bitcoin USD
Total Value (CAD): $4,324.48
1-Year Return: 127.74%
----------------------------------------------------------------------------------
Ticker Name: Vanguard S&P 500 Index ETF
Total Value (CAD): $3,250.72
1-Year Return: 38.94%
----------------------------------------------------------------------------------


Ticker Name: Netflix, Inc.
Total Value (CAD): $2,172.64
1-Year Return: 102.77%
----------------------------------------------------------------------------------


Ticker Name: Taiwan Semiconductor Manufacturing Company Limited
Total Value (CAD): $1,685.57
1-Year Return: 114.86%
----------------------------------------------------------------------------------


Ticker Name: Modine Manufacturing Company
Total Value (CAD): $1,546.10
1-Year Return: 163.67%
----------------------------------------------------------------------------------


...

Ticker Name: Nutrien Ltd.
Total Value (CAD): $217.00
1-Year Return: -6.25%
----------------------------------------------------------------------------------


Ticker Name: The Procter & Gamble Company
Total Value (CAD): $215.18
1-Year Return: 22.93%
----------------------------------------------------------------------------------


Ticker Name: Adobe Inc.
Total Value (CAD): $209.80
1-Year Return: -10.75%
----------------------------------------------------------------------------------
Ticker Name: Sprott Uranium Miners ETF
Total Value (CAD): $202.55
1-Year Return: 1.72%
----------------------------------------------------------------------------------


Ticker Name: Kinross Gold Corporation
Total Value (CAD): $188.73
1-Year Return: 79.35%
----------------------------------------------------------------------------------


Ticker Name: Rolls-Royce Holdings plc
Total Value (CAD): $127.43
1-Year Return: 112.04%
----------------------------------------------------------------------------------
Ticker Name: Sprott Physical Gold Trust
Total Value (CAD): $69.07
1-Year Return: 28.52%
----------------------------------------------------------------------------------


In [ ]:
df_data_updated.drop(columns=['Currency', 'Current Price', 'Total Value (Base)', 'Logo URL'])

,Ticker,Ticker Name,Total Value (CAD)
22,RKLB,"Rocket Lab USA, Inc.","$7,866.06"
31,BTC-USD,Bitcoin USD,"$4,324.48"
29,VFV.TO,Vanguard S&P 500 Index ETF,"$3,250.72"
17,NFLX,"Netflix, Inc.","$2,172.64"
25,TSM,Taiwan Semiconductor Manufacturing Company Lim...,"$1,685.57"
15,MOD,Modine Manufacturing Company,"$1,546.10"
5,COST,Costco Wholesale Corporation,"$1,206.83"
19,NVDA,NVIDIA Corporation,"$1,120.70"
16,MSFT,Microsoft Corporation,"$1,031.58"
26,U,Unity Software Inc.,"$1,012.43"
